# Tutorial

## Monte Carlo Simulation

## for American Options

(c) 2017 Martin Guggisberg

## ToDo

1. [ X ] relevant literature
1. [   ] one dimensional Wiener process
1. [   ] multi dimensional Wiener process
1. [   ] Regression -> estimating density probability function
1. [   ] Regression: using different basis functions
1. [   ] standard SDE Discretization 
1. [   ] using different SDE discretizations
1. [   ] comparing results with experiments from literature
1. [   ] using simulated anealing for regression
1. [   ]

## literature

### articles
    - XX

### books
    - xx